In [17]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os
import glob
import PIL
from PIL import Image
from scipy.misc import toimage
from torch.utils import data as D
import random

In [21]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

In [19]:
input_size = 28*28
latent_size = 64
hidden_size = 256

num_epochs = 200
batch_size = 100

sample_dir = "./result_gan/"

if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [29]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
                                     std=(0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

In [30]:
class Generator(nn.Module):
    def __init__(self, input_nodes, latent_nodes, hidden_nodes):
        super(Generator, self).__init__()
        self.linear1 = nn.Linear(latent_nodes, hidden_nodes)
        self.linear2 = nn.Linear(hidden_nodes, hidden_nodes)
        self.linear3 = nn.Linear(hidden_nodes, input_nodes)
        
    def forward(self, x):
        x = (F.relu(self.linear1(x)))
        x = (F.relu(self.linear2(x)))
        x = (F.tanh(self.linear3(x)))
        return x

In [31]:
class Discriminator(nn.Module):
    def __init__(self, input_nodes, hidden_nodes):
        super(Discriminator, self).__init__()
        self.linear1 = nn.Linear(input_nodes, hidden_nodes)
        self.linear2 = nn.Linear(hidden_nodes, hidden_nodes)
        self.linear3 = nn.Linear(hidden_nodes, 1)
        
    def forward(self, x):
        x = (F.leaky_relu(self.linear1(x), 0.2))
        x = (F.leaky_relu(self.linear2(x), 0.2))
        x = (F.sigmoid(self.linear3(x)))
        return x

In [33]:
G = Generator(input_size, latent_size, hidden_size)
D = Discriminator(input_size, hidden_size)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

G.to(device)
D.to(device)

cuda:0


Discriminator(
  (linear1): Linear(in_features=784, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=256, bias=True)
  (linear3): Linear(in_features=256, out_features=1, bias=True)
)

In [34]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [35]:
# Start training
total_step = len(trainloader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(trainloader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        # ================================================================== #
        #                        Train the discriminator                     #
        # ================================================================== #

        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        d_loss = d_loss_real + d_loss_fake
        
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()

        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        g_loss = criterion(outputs, real_labels)
        
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# Save the model checkpoints 
torch.save(G.state_dict(), sample_dir + '/G.ckpt')
torch.save(D.state_dict(), sample_dir + '/D.ckpt')

Epoch [0/200], Step [200/600], d_loss: 0.0278, g_loss: 4.2495, D(x): 1.00, D(G(z)): 0.02
Epoch [0/200], Step [400/600], d_loss: 0.1151, g_loss: 5.8789, D(x): 0.97, D(G(z)): 0.08
Epoch [0/200], Step [600/600], d_loss: 0.0757, g_loss: 4.5425, D(x): 0.96, D(G(z)): 0.04
Epoch [1/200], Step [200/600], d_loss: 0.0600, g_loss: 4.9002, D(x): 0.99, D(G(z)): 0.05
Epoch [1/200], Step [400/600], d_loss: 0.0890, g_loss: 3.9877, D(x): 0.94, D(G(z)): 0.02
Epoch [1/200], Step [600/600], d_loss: 0.1364, g_loss: 4.2424, D(x): 0.95, D(G(z)): 0.06
Epoch [2/200], Step [200/600], d_loss: 0.4604, g_loss: 4.3579, D(x): 0.78, D(G(z)): 0.05
Epoch [2/200], Step [400/600], d_loss: 0.1148, g_loss: 4.8738, D(x): 0.98, D(G(z)): 0.09
Epoch [2/200], Step [600/600], d_loss: 0.1526, g_loss: 4.7086, D(x): 0.96, D(G(z)): 0.07
Epoch [3/200], Step [200/600], d_loss: 0.5600, g_loss: 1.8924, D(x): 0.82, D(G(z)): 0.20
Epoch [3/200], Step [400/600], d_loss: 0.3438, g_loss: 3.4668, D(x): 0.90, D(G(z)): 0.11
Epoch [3/200], Step [